<a href="https://colab.research.google.com/github/FedericoRaschiatore0123/GATr_Deep_Learning_Project/blob/main/GA_sample_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import numpy as np
import h5py
import os
import pandas as pd
import seaborn as sns

In [6]:
# create a dictionary of global variables
global_variables = {

    'dim_GA': 16
}

In [7]:
def embed_pos_mv_16(pos):
    """
    Embeds points in 3D space into multivectors of dimension 16 using an alternative approach.

    Args:
        pos (torch.Tensor): tensor of input points with shape (n_elements, 3)

    Returns:
        mv torch.Tensor: tensor of multivectors of dimension 16 with shape (1, n_elements, 1, 16)
    """
    # Get the number of points and the dimensionality
    n_elements, dim = pos.shape

    # Create multivector tensor
    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 14] = 1 # homogeneous component, e_{123}
    multivector[:, 11] = pos[:, 0] # x, e_{023}
    multivector[:, 12] = pos[:, 1] # y, e_{013}
    multivector[:, 13] = pos[:, 2] # z, e_{012}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv


# ???: question about sign convention
# sign convention chosen as indicated in the paper
# the choice of the sign allow to give the correct geometric interpretation
#       to the components. Important to have the convention be consistent
#       for the whole embedding

In [8]:
# check function!
pts = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15],
             [16, 17, 18], [19, 20, 21], [22, 23, 24], [25, 26, 27], [28, 29, 30]]
pts_tensor = torch.tensor(pts)
mv = embed_pos_mv_16(pts_tensor)
print(mv) # ok!

tensor([[[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  3.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  5.,  6.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  8.,  9.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 11., 12.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 13., 14., 15.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16., 17., 18.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 19., 20., 21.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 22., 23., 24.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25., 26., 27.,
            1.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,

In [9]:
def embed_face_mv_16(face):
    """
    Embeds the faces into multivectors of dimension 16

    Input:
        face torch.Tensor of size (n_elements, 3)
    Output:
        mv torch.Tensor of size (1, n_elements, 1, 16)
    """

    # Get the number of points and the dimensionality
    n_elements, dim = face.shape

    # Create multivector tensor
    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 2] = face[:, 0] # e_{1}
    multivector[:, 3] = face[:, 1] # e_{2}
    multivector[:, 4] = face[:, 2] # e_{3}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv

In [10]:
def embed_wss_mv_16(wss):


    # it can be embedded as a translation in the G_{3,0,1} space

    n_elements, dim = wss.shape

    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 0] = 1
    multivector[:, 5] = 0.5*wss[:, 0] # e_{01}
    multivector[:, 6] = 0.5*wss[:, 1] # e_{02}
    multivector[:, 7] = 0.5*wss[:, 2] # e_{03}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv

In [11]:
def embed_inlet_mv_16(inlet):
    """
    Embed the inlet torch tensor as a 16 dimensional multivector
    Input:
        inlet torch.Tensor of size (n_elements, 1)
    Output:
        mv torch.Tensor of size (1, n_elements, 1, 1)
    """
    n_elements = inlet.shape[0]

    multivector = torch.zeros(n_elements, global_variables['dim_GA'])
    multivector[:, 0] = inlet[:,0]

    mv = multivector.reshape(1, n_elements, 1, global_variables['dim_GA'])

    return mv